In [1]:
%matplotlib inline

import numpy as np
import collections
from collections import OrderedDict, Counter, defaultdict
import pandas as pd

import Bio
from Bio import SeqIO
from Bio.Seq import Seq

import seaborn as sns
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, show 
import matplotlib.patches as patches    
from matplotlib import colors


import glob

import subprocess
from subprocess import call

import re

import pickle

from Bio.SubsMat import MatrixInfo
import Bio.Data.CodonTable

from scipy import stats


from Bio.SubsMat import MatrixInfo
import Bio.Data.CodonTable

import joypy

import pyBigWig


Bad key text.latex.unicode in file /home/alla/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.frameon in file /home/alla/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 423 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key pgf.debug in file /home/alla/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 444 ('pgf.debug           : False')
You probably need to get an updated matplotlibrc file from
https://githu

# Metadata 

In [6]:
path = '/home/alla/make_PhD_great_again/NTE/VERSION_since_May2021/'

# gencode 25
metadata_pc_g25 = pd.read_csv(path+'tmp_res/metadata_pc_g25.txt', sep='\t')
metadata_pc_g25['tr_id1'] = [x.split('.')[0] for x in metadata_pc_g25['tr_id'].tolist()] 

# Overlaps with coding exons 

In [15]:
ovlp = pd.read_csv('/home/alla/make_PhD_great_again/SNPS_in_NTE/GATK_CodALignVIew_McDonald_Kreitman/tmp_res/master_df_top5000_ext_no_ovlps_g25_39_refseq.txt', sep='\t')
ovlp['tr_id1'] = [x.split('.')[0] for x in ovlp['tr_id'].tolist()]

print (ovlp.shape[0])

ovlp[0:3]

4449


,tr_id,strand,global_coo,tr_id1,gene,pos_of_codon,cds_start_pos,chr,Start_ext,Stop_ext,global_rank_ext,Start_codon_ext,Proteomics_count_ext,Coverage_value_ext,CDS_ratio,Riboseq_Summary
0,ENST00000001146.6,-,chr2:72147835-72147954,ENST00000001146,CYP26B1,84,204,chr2,85,205,3888,CTG,0.0,11.428571,0.0,CTG; 85-205; Rank: 3888; cov: 11.43
1,ENST00000003084.10,+,chr7:117480044-117480094,ENST00000003084,CFTR,81,132,chr7,82,133,3401,AGG,0.0,16.666667,0.0,AGG; 82-133; Rank: 3401; cov: 16.67
2,ENST00000003302.8,-,chr11:113875502-113875561,ENST00000003302,USP28,9,69,chr11,10,70,2063,AGG,0.0,40.000000,0.0,AGG; 10-70; Rank: 2063; cov: 40.0


# Genomic coo of 5'UTR + start codon (ATG) + 4nt for the context

In [16]:
tmp = metadata_pc_g25[(metadata_pc_g25['utr5_end'].notna())]
tmp2 = tmp[tmp['utr5_end'] > 0]

tmp2['tr_len'] = tmp2['transcript_seq'].str.len()
tmp3 = tmp2[tmp2['tr_len'] > tmp2['cds_stop_pos']]

tmp4 = tmp3[tmp3['cds_stop_codon'].isin(['TAG', 'TGA', 'TAA'])]

tmp5_only_AUG = tmp4[tmp4['cds_start_codon'] == 'ATG']

print (metadata_pc_g25.shape[0], tmp2.shape[0], tmp3.shape[0], tmp4.shape[0], tmp5_only_AUG.shape[0])

94359 78873 60109 60065 59948


/home/alla/.local/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [21]:
!mkdir -p tmp_res

In [22]:
tmp4[['tr_id', 'cds_start_pos']].to_csv('tmp_res/utr5_cdsATG_4nt_local.txt', sep='\t', index=False)

In [24]:
def prepare_global_coo(path_to_file_with_global_coo, colname):
    global_coo_g25 = pd.read_csv(path_to_file_with_global_coo, sep='\t')
    global_coo_g25_f = global_coo_g25[global_coo_g25['hit'] == True]
    global_coo_g25_f = global_coo_g25_f.sort_values(by=['seqnames', 'start'])
    
    global_coo_g25_f['global_coo'] = global_coo_g25_f['seqnames']+':'+global_coo_g25_f['start'].astype(str) +'-'+global_coo_g25_f['end'].astype(str)
    global_coo_g25_f = global_coo_g25_f[['group_name', 'strand', 'global_coo']].groupby(['group_name', 'strand']).agg('+'.join).reset_index()
    global_coo_g25_f.columns = ['tr_id', 'strand', colname]
    return global_coo_g25_f

In [25]:
primary_global = prepare_global_coo(path_to_file_with_global_coo='tmp_res/utr5_cdsATG_4nt_global.txt',
                                   colname = 'global_coo')

In [27]:
primary_global.to_csv('tmp_res/ATG_transcripts_5UTR_ATG_4nt.txt', 
                      sep='\t', 
                      index=False)

# Genomic coo of 5'UTR + start codon (nonATG) + dAUG1 + 4nt of context 

In [ ]:
d = {''}

In [ ]:
18844

# Functions for parsing alignments 

In [28]:
def get_aln(fasta_input):
    aln_col = []
    ids_li = []
    
    with open(fasta_input, 'r') as input_handle:
        for record in SeqIO.parse(input_handle, "fasta"):
            seq = str(record.seq)
            ids = str(record.id)
            aln_col.append([seq[i:i+1] for i in range(0, len(seq), 1)])
            ids_li.append(ids)
    return aln_col, ids_li

def re_index_with_gaps(aln_col):
    new_col_ind = []

    # take ref seq
    counter_pos = 0 # to exclude '-'

    for col in pd.DataFrame(aln_col).to_numpy()[0]:
        if (col == '-') | (col == '.'): # UCC UGA C = TCC TGA C
            new_col_ind.append(counter_pos)
        else: 
            counter_pos += 1
            new_col_ind.append(counter_pos)
    
    return new_col_ind


def slice_aln(aln_col, new_col_ind, a, b):
    fr_stop_motif_array = []
    for row in pd.DataFrame(aln_col).to_numpy():
        tmp = []
        for ind, col in zip(new_col_ind, row):
            if (ind >= a) & (ind <= b):
                tmp.append(col)
            
        fr_stop_motif_array.append(tmp)
    
    return fr_stop_motif_array
